In [ ]:
from cmp import *
% matplotlib notebook
np.set_printoptions(threshold=np.nan)

In [ ]:
# Inputs
eq = np.isclose
# Lattice vectors (3 vectors of length 3)
a = 1
b = 2
a1 = np.array([1, 0, 0])
a2 = np.array([0, 1, 0])
a3 = np.array([0, 0, 1])
theta = 80*np.pi/180

# Array of basis vectors
basis = np.array([[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# Colors for each of the basis vectors
colors = ('xkcd:cement','b','b','b')
# Size multiplier for each of the atoms. Default is 1
sizes = (2,1,1,1)
verbose = True


# Gridline type:
# Soft: Lines along cartesian axes. Takes into account nonequal lattice spacing
# LatticeVectors: Lines along the latticevectors (only on lattice points)
GridType = "lattice"

# Limit type:
# individual: Sets the limits as max(nx*a1,ny*a2,nz*a3), so we include nx unitcells in the a1 direction, etc.
# sum: Sets the limits r_min = n_min*[a1 a2 a3] and likewise for n_max
LimType = "dynamic"
Maxs = [2,2,2]
Mins = [0,0,0]

LatticeType = "base centred cubic"
#LatticeCreator(a1,a2,a3,basis,colors,sizes,LimType,GridType,Mins,Maxs)
LatticeCreator(Lattice = LatticeType, verbose=False)

In [ ]:
def LatticePlotter(a1, a2, a3, AtomicPositions, AtomicColors, AtomicSizes,
                   LatticePosition, LatticeCoefficients, n_max, GridType, r_min, r_max, verbose=False):
    """
    Takes the input lattice, adds gridlines and plots everything
    """
    # Create the figure
    fig = plt.figure()
    ax = fig.gca(projection="3d")

    # Plot atoms. For now a single size and color
    ax.scatter(AtomicPositions[:, 0], AtomicPositions[:, 1],
               AtomicPositions[:, 2], c=AtomicColors, s=AtomicSizes)

    # Get the relevant gridlines:
    g_col = 'k'
    g_w = 0.5
    pruned_lines = CreateLines(AtomicPositions[LatticePosition], a1, a2, a3, n_max, LatticeCoefficients[LatticePosition])
    for line in pruned_lines:
        ax.plot(line[0], line[1], line[2], color=g_col, linewidth=g_w)
    # plot lattice vectors
    ax.quiver(0, 0, 0, a1[0], a1[1], a1[2])
    ax.quiver(0, 0, 0, a2[0], a2[1], a2[2])
    ax.quiver(0, 0, 0, a3[0], a3[1], a3[2])
    ax.text(a1[0] / 2, a1[1] / 2, a1[2] / 2, '$a_1$')
    ax.text(a2[0] / 2, a2[1] / 2, a2[2] / 2, '$a_2$')
    ax.text(a3[0] / 2, a3[1] / 2, a3[2] / 2, '$a_3$')

    # Set limits, orthographic projection (so we get the beautiful hexagons),
    # no automatic gridlines, and no axes
    ax.set_aspect('equal')
    ax.set_proj_type('ortho')
    ax.set_xlim3d([r_min[0], r_max[0]])
    ax.set_ylim3d([r_min[1], r_max[1]])
    ax.set_zlim3d([r_min[2], r_max[2]])
    ax.grid(False)
    plt.axis('equal')
    plt.axis('off')

    # make the panes transparent (the plot box)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    plt.show()


In [ ]:
(a1,a2,a3), basis = LatticeChooser("bcc")
r_min,r_max,n_min,n_max = FindLimits("sum",a1,a2,a3)
colors = ["xkcd:cement"]
sizes = [2]
N_basis = 1
(AtomicPositions, LatticeCoefficients, AtomicColors, AtomicSizes,
     LatticePosition) = LatticeGenerator(a1, a2, a3, basis, colors, sizes,
                                         LimType, n_min, n_max, r_min, r_max,
                                         N_basis)

pruned_lines = GridLines(a1,a2,a3,AtomicPositions,colors,sizes,LatticePosition,"soft",r_min,r_max)

In [ ]:
def CreateLines(points, vectors):
    """
    Creates lines along vectors and limits these to the given plot box
    """
    
    # Creating the gridlines along latticevectors
    # for each lattice point, we calculate the cosine of the angle between each of the lattice vectors and each of the separation vectors to other lattice points
    # This allows us to determine which atoms lie along a lattice vector from a given point
    # Then we can just pick the atom furthest away (in the "positive" direction), and use it as the end point for the grid line.
    lines = []

    # Create all gridlines needed and append them to the lines-list
    numPoints = np.shape(points)[0]
    
    for rowID in range(numPoints):
        CurrentPoint = points[rowID, ]
        for vec in vectors:
            # First we want to delete the origin from the separation
            sep = points - CurrentPoint
            # We make a boolean array of values equal to 0, sum them up and if a row equals 3, then it is the null vector
            noughts = np.sum((sep == np.array([0,0,0])),1) == 3
            # Then we select all the separation vectors that are not the nullvector
            sep = sep[noughts != True]
            nonpoints = points[noughts != True]
            mag_sep = mag(sep)
            
            # We calculate the cosine of the angle between the current lattice vector and the separation vectors
            cosine = sep.dot(vec)/(mag_sep*mag(vec))
            # If it is close to 1, then the vectors are parallel
            para = eq(1,cosine)
            # We get the parallel points and magnitude of parallel separation vectors
            parapoints = nonpoints[para,]
            magparasep = mag_sep[para]
            if magparasep.shape[0] == 0:
                # if there are no points further out, that have a parallel separation vector, we just pass, as we are at the edge of the lattice
                pass
            else:
                # We pick the parallel point, that has the largest 
                endpointbool = magparasep == np.amax(magparasep)
                endpoint = parapoints[endpointbool]
                endpoint = np.squeeze(endpoint)
                rawline = np.vstack((CurrentPoint,endpoint))
                lines.append([rawline[:,0], rawline[:,1], rawline[:,2]])
            
            
    return lines

In [ ]:
lines = CreateLines(AtomicPositions,np.array([a1,a2,a3]))
fig = plt.figure()
ax = fig.gca(projection="3d")
for line in lines:
    ax.plot(line[0],line[1],line[2],color='k',linewidth=0.5)

In [ ]:
# Creating the gridlines along latticevectors
# for each lattice point, we calculate the cosine of the angle between each of the lattice vectors and each of the separation vectors to other lattice points
# This allows us to determine which atoms lie along a lattice vector from a given point
# Then we can just pick the atom furthest away (in the "positive" direction), and use it as the end point for the grid line.

l = np.array([[1,0,0],[0,1,0],[0,0,1]])
p = np.zeros((4**3,3))
counter = 0
for i in range(4):
    for j in range(4):
        for k in range(4):
            p[counter,] = np.array((i,j,k))
            counter += 1

sep1 = p-l[0,]
sep2 = p-l[1,]
sep3 = p-l[2,]
a1,a2,a3 = l
mag_sep1 = mag(sep1)
dot1= sep1.dot(a1)
para = eq(1,(dot1/(mag_sep1*mag(a1))))
paraPoints = p[para,]
paraSep = sep1[para,]
magparaSep = mag(paraSep)
maxsep = np.amax(magparaSep)
maxsepBool = magparaSep == maxsep
endpoint = paraPoints[maxsepBool,]


In [ ]:
LatticePlotter(a1, a2, a3, AtomicPositions, AtomicColors, AtomicSizes,
                   LatticePosition, LatticeCoefficients, n_max, "lattice", r_min, r_max)

In [ ]:
point=a1+a2+a3
coefficient=np.array([1,1,1])
endpoint = np.array([a1,a2,a3]).T @ (n_max-coefficient)
print(endpoint)
print(point)